In [1]:
from pathlib import Path

import numpy as np
from neuroglancer_scripts.dyadic_pyramid import fill_scales_for_dyadic_pyramid
from numpy import ndarray
from skimage.transform import resize
from zimg import ZImg, ZImgInfo

from nii_2_precomputed import Resolution, build_base_json_dict
from util import pretty_print_object

In [2]:
image_path = Path(r"D:\EEG Data\nii\20230530\full16_100um_2009b_sym.nii.gz")
image_info: ZImgInfo = ZImg.readImgInfos(str(image_path))[0]

In [3]:
target_y = 175
ratio = image_info.height / target_y
scaled_image_size = Resolution(
    x=round(image_info.width / ratio), y=target_y, z=round(image_info.depth / ratio)
)
scaled_step_size = 20
scaled_ranges = [
    (scaled_start, min(scaled_start + scaled_step_size, scaled_image_size.z))
    for scaled_start in range(0, scaled_image_size.z, scaled_step_size)
]
image_starts = [round(scaled_start * ratio) for scaled_start, _ in scaled_ranges]
image_ends = image_starts[1:] + [image_info.depth]

pretty_print_object(scaled_ranges)
pretty_print_object(list(zip(image_starts, image_ends)))

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ [(0, 20), (20, 40), (40, 60), (60, 80), (80, 100), (100, 120), (120, 140), (140, 142)]                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ [(0, 266), (266, 533), (533, 799), (799, 1065), (1065, 1331), (1331, 1598), (1598, 1864), (1864, 1890)]         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [4]:
info = {
    "type": "image",
    "data_type": image_info.dataTypeString(),
    "num_channels": 1,
    "scales": [
        {
            "chunk_sizes": [],
            "encoding": "raw",
            "sharding": {
                "@type": "neuroglancer_uint64_sharded_v1",
                "hash": "identity",
                "minishard_bits": 6,
                "minishard_index_encoding": "gzip",
                "data_encoding": "gzip",
                "preshift_bits": 9,
                "shard_bits": 15,
            },
            "resolution": [1250000.0] * 3,
            "size": list(scaled_image_size),
            "voxel_offset": [0, 0, 0],
        }
    ],
}
fill_scales_for_dyadic_pyramid(info, target_chunk_size=64)
pretty_print_object(info)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ {                                                                                                               │
│     'type': 'image',                                                                                            │
│     'data_type': 'uint16',                                                                                      │
│     'num_channels': 1,                                                                                          │
│     'scales': [                                                                                                 │
│         {                                                                                                       │
│             'chunk_sizes': [[64, 64, 64]],                                                                      │
│             'encoding': 'raw',                                                                                  │
│             'sharding': {                                                                                       │
│                 '@type': 'neuroglancer_uint64_sharded_v1',                                                      │
│                 'hash': 'identity',                                                                             │
│                 'minishard_bits': 6,                                                                            │
│                 'minishard_index_encoding': 'gzip',                                                             │
│                 'data_encoding': 'gzip',                                                                        │
│                 'preshift_bits': 9,                                                                             │
│                 'shard_bits': 15                                                                                │
│             },                                                                                                  │
│             'resolution': [1250000.0, 1250000.0, 1250000.0],                                                    │
│             'size': [148, 175, 142],                                                                            │
│             'voxel_offset': [0, 0, 0],                                                                          │
│             'key': '1mm'                                                                                        │
│         },                                                                                                      │
│         {                                                                                                       │
│             'chunk_sizes': [[64, 64, 64]],                                                                      │
│             'encoding': 'raw',                                                                                  │
│             'sharding': {                                                                                       │
│                 '@type': 'neuroglancer_uint64_sharded_v1',                                                      │
│                 'hash': 'identity',                                                                             │
│                 'minishard_bits': 6,                                                                            │
│                 'minishard_index_encoding': 'gzip',                                                             │
│                 'data_encoding': 'gzip',                                                                        │
│                 'preshift_bits': 9,                                                                             │
│                 'shard_bits': 15                                                                                │
│             },                                        

In [5]:
def save_numpy_array(array: ndarray, directory: Path, name_prefix: str) -> None:
    shape_name = "_".join(str(dim_length) for dim_length in array.shape)
    save_path = directory / f"{name_prefix}_{shape_name}.npz"
    np.savez_compressed(save_path)


def change_resized_chunk_dtype(array: ndarray, target_dtype: np.dtype) -> ndarray:
    if target_dtype == np.float64:
        return array
    if target_dtype == np.float32:
        return array.astype(np.float32)
    if target_dtype.kind == 'u':
        return (array * np.iinfo(target_dtype).max).astype(target_dtype)
    raise ValueError('unknown target dtype')

In [6]:
image_data_obj = ZImg(str(image_path))
image_data = image_data_obj.data[0][0].transpose()

resized_data = np.empty(scaled_image_size, dtype=image_data.dtype)
for i, image_range in enumerate(zip(image_starts, image_ends)):
    image_data_chunk = image_data[:, :, image_range[0]: image_range[1]]
    scaled_range = scaled_ranges[i]
    resized_chunk = resize(
        image_data_chunk,
        (scaled_image_size.x, scaled_image_size.y, scaled_range[1] - scaled_range[0]),
        anti_aliasing=True,
    )
    result_chunk = change_resized_chunk_dtype(resized_chunk, image_data.dtype)
    resized_data[:, :, scaled_range[0]: scaled_range[1]] = result_chunk

save_numpy_array(resized_data, Path(r"C:\Workspace"), "resized")

In [7]:
from nii_2_precomputed import convert_to_tensorstore_scale_metadata, open_tensorstore
import json
import tensorstore as ts

base_json_dict = build_base_json_dict(image_data_obj.info, Resolution(1250000, 1250000, 1250000), scaled_image_size,
                                      image_data.dtype, 'http://localhost:8080/')
out_folder = Path(r'C:\Workspace\fuck')
pretty_print_object(base_json_dict, "base.json")
with open(out_folder / "base.json", "w") as base_json_file:
    json.dump(base_json_dict, base_json_file, indent=2)

multiscale_metadata = {
    "data_type": info["data_type"],
    "num_channels": info["num_channels"],
    "type": info["type"],
}
# for i, scale in enumerate(info['scales']):
#     store = open_tensorstore(f"channel_{i}", out_folder, convert_to_tensorstore_scale_metadata(scale), multiscale_metadata)
#     store[ts.d["channel"][0]] = resized_data
store = open_tensorstore("channel_0", out_folder, convert_to_tensorstore_scale_metadata(info['scales'][0]),
                         multiscale_metadata)
store[ts.d['channel'][0]] = resized_data

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│    2 import json                                                                                 │
│    3 import tensorstore as ts                                                                    │
│    4                                                                                             │
│ ❱  5 base_json_dict = build_base_json_dict(image_data_obj.info, Resolution(1250000, 1250000,     │
│    6 │   │   │   │   │   │   │   │   │     image_data.dtype, 'http://localhost:8080/')           │
│    7 out_folder = Path(r'C:\Workspace\fuck')                                                     │
│    8 pretty_print_object(base_json_dict, "base.json")                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'build_base_json_dict' is not defined